In [1]:
import ee
import geemap

In [2]:
# Authentication and initialization
ee.Authenticate()
ee.Initialize(project="alessiodeluca")

# Define the region of interest
geometry = ee.Geometry.Point([16.563798, 43.496451])
roi = geometry.buffer(10000)

# Load Sentinel-2 images for the specified dates
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                .filterBounds(geometry) \
                .filterDate('2017-07-06', '2017-08-10')


In [3]:

# Estract images
imageList = collection.toList(collection.size())
pre_fire_image = ee.Image(imageList.get(0)).select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'])
post_fire_image = ee.Image(imageList.get(3)).select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'])


## Cascade Kmeans Clustering

In [4]:
# Function to perform clustering using Cascade KMeans
def perform_cascade_clustering(image, min_clusters, max_clusters, restarts=10, manual=False, init=False, distance_function="Euclidean", max_iterations=None):
    # Create a training dataset by sampling the image
    training = image.sample(region=roi, scale=10, numPixels=5000)

    # Instantiate and train the clusterer with Cascade KMeans
    clusterer = ee.Clusterer.wekaCascadeKMeans(minClusters=min_clusters,
                                               maxClusters=max_clusters,
                                               restarts=restarts,
                                               manual=manual,
                                               init=init,
                                               distanceFunction=distance_function,
                                               maxIterations=max_iterations).train(training)

    # Apply the clusterer to the image
    clustered_image = image.cluster(clusterer)

    return clustered_image

# Define parameters for Cascade KMeans clustering
min_clusters = 5
max_clusters = 7
restarts = 10
manual = False
init = False
distance_function = "Euclidean"
max_iterations = None

# Perform clustering for pre-fire and post-fire images using Cascade KMeans
pre_fire_clustered = perform_cascade_clustering(pre_fire_image, min_clusters, max_clusters, restarts, manual, init, distance_function, max_iterations)
post_fire_clustered = perform_cascade_clustering(post_fire_image, min_clusters, max_clusters, restarts, manual, init, distance_function, max_iterations)

# Visualization of results
Map = geemap.Map()
Map.centerObject(roi, 10)

# Add original and clustered image layers to the map
Map.addLayer(pre_fire_clustered.randomVisualizer(), {}, 'Pre-Fire Clusters')
Map.addLayer(post_fire_clustered.randomVisualizer(), {}, 'Post-Fire Clusters')

# Display the map
Map


Map(center=[43.49646125887293, 16.56379839217765], controls=(WidgetControl(options=['position', 'transparent_b…

## KMeans Clustering

In [5]:
# Function to perform clustering
def perform_clustering(image, num_clusters):
    # Create a training dataset by sampling the image
    training = image.sample(region=roi, scale=10, numPixels=5000)

    # Instantiate and train the clusterer
    clusterer = ee.Clusterer.wekaKMeans(num_clusters).train(training)

    # Apply the clusterer to the image
    clustered_image = image.cluster(clusterer)

    return clustered_image

num_clusters = 4

# Perform clustering for pre-fire, during-fire, and post-fire images
pre_fire_clustered = perform_clustering(pre_fire_image, num_clusters)
post_fire_clustered = perform_clustering(post_fire_image, num_clusters)

# Visualization of results
Map = geemap.Map()
Map.centerObject(roi, 10)

# Add original and clustered image layers to the map
Map.addLayer(pre_fire_clustered.randomVisualizer(), {}, 'Pre-Fire Clusters')
Map.addLayer(post_fire_clustered.randomVisualizer(), {}, 'Post-Fire Clusters')

# Display the map
Map


Map(center=[43.49646125887293, 16.56379839217765], controls=(WidgetControl(options=['position', 'transparent_b…

## LVQ Clustering

In [6]:
# Function to perform clustering using Learning Vector Quantization (LVQ)
def perform_lvq_clustering(image, num_clusters, learning_rate=1.0, epochs=1000, normalize_input=False):
    # Create a training dataset by sampling the image
    training = image.sample(region=roi, scale=10, numPixels=5000)

    # Instantiate and train the clusterer with LVQ
    clusterer = ee.Clusterer.wekaLVQ(numClusters=num_clusters,
                                     learningRate=learning_rate,
                                     epochs=epochs,
                                     normalizeInput=normalize_input).train(training)

    # Apply the clusterer to the image
    clustered_image = image.cluster(clusterer)

    return clustered_image

# Define parameters for LVQ clustering
num_clusters = 4  # Number of clusters
learning_rate = 0.5  # Learning rate (between 0 and 1)
epochs = 1000  # Number of epochs
normalize_input = True  # Normalize input data

# Perform clustering for pre-fire and post-fire images using LVQ
pre_fire_clustered = perform_lvq_clustering(pre_fire_image, num_clusters, learning_rate, epochs, normalize_input)
post_fire_clustered = perform_lvq_clustering(post_fire_image, num_clusters, learning_rate, epochs, normalize_input)

# Visualization of results
Map = geemap.Map()
Map.centerObject(roi, 10)

# Add original and clustered image layers to the map
Map.addLayer(pre_fire_clustered.randomVisualizer(), {}, 'Pre-Fire Clusters')
Map.addLayer(post_fire_clustered.randomVisualizer(), {}, 'Post-Fire Clusters')

# Display the map
Map


Map(center=[43.4964612588728, 16.563798392177652], controls=(WidgetControl(options=['position', 'transparent_b…

## XMeans Clustering

In [7]:
# Function to perform clustering using X-Means
def perform_xmeans_clustering(image, min_clusters, max_clusters, max_iterations=3, max_kmeans=1000, max_for_children=1000, use_kd=False, cutoff_factor=0.0, distance_function="Euclidean", seed=10):
    # Create a training dataset by sampling the image
    training = image.sample(region=roi, scale=10, numPixels=5000)

    # Instantiate and train the clusterer with X-Means
    clusterer = ee.Clusterer.wekaXMeans(minClusters=min_clusters,
                                        maxClusters=max_clusters,
                                        maxIterations=max_iterations,
                                        maxKMeans=max_kmeans,
                                        maxForChildren=max_for_children,
                                        useKD=use_kd,
                                        cutoffFactor=cutoff_factor,
                                        distanceFunction=distance_function,
                                        seed=seed).train(training)

    # Apply the clusterer to the image
    clustered_image = image.cluster(clusterer)

    return clustered_image

# Define parameters for X-Means clustering
min_clusters = 2  # Minimum number of clusters
max_clusters = 8  # Maximum number of clusters
max_iterations = 3  # Maximum number of total iterations
max_kmeans = 1000  # Maximum number of iterations for K-Means
max_for_children = 1000  # Maximum number of iterations for K-Means on child centroids
use_kd = False  # Use KD-Tree
cutoff_factor = 0.0  # Percentage of centroids split if none of the children win
distance_function = "Euclidean"  # Distance function
seed = 10  # Seed for randomization

# Perform clustering for pre-fire and post-fire images using X-Means
pre_fire_clustered = perform_xmeans_clustering(pre_fire_image, min_clusters, max_clusters, max_iterations, max_kmeans, max_for_children, use_kd, cutoff_factor, distance_function, seed)
post_fire_clustered = perform_xmeans_clustering(post_fire_image, min_clusters, max_clusters, max_iterations, max_kmeans, max_for_children, use_kd, cutoff_factor, distance_function, seed)

# Visualization of results
Map = geemap.Map()
Map.centerObject(roi, 10)

# Add original and clustered image layers to the map
Map.addLayer(pre_fire_clustered.randomVisualizer(), {}, 'Pre-Fire Clusters')
Map.addLayer(post_fire_clustered.randomVisualizer(), {}, 'Post-Fire Clusters')

# Display the map
Map


Map(center=[43.49646125887293, 16.56379839217765], controls=(WidgetControl(options=['position', 'transparent_b…